d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 1200px">
</div>

# Intro To DataFrames, Part #4

**Technical Accomplishments:**
* Create a User Defined Function (UDF)
* Execute a join operation between two `DataFrames`

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup<br>

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [0]:
%run "../Includes/Classroom-Setup"

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) The Data Source

This data uses the **Pageviews By Seconds** data set.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# I've already gone through the exercise to determine
# how many partitions I want and in this case it is...
partitions = 8

# Make sure wide operations don't repartition to 200
spark.conf.set("spark.sql.shuffle.partitions", str(partitions))

In [0]:
(source, sasEntity, sasToken) = getAzureDataSource()
spark.conf.set(sasEntity, sasToken)

parquetFile = source + "/wikipedia/pageviews/pageviews_by_second.parquet/"

In [0]:
# Create our initial DataFrame. We can let it infer the 
# schema because the cost for parquet files is really low.
pageviewsDF = (spark.read
  .option("inferSchema", "true")                # The default, but not costly w/Parquet
  .parquet(parquetFile)                         # Read the data in
  .repartition(partitions)                      # From 7 >>> 8 partitions
  .withColumnRenamed("timestamp", "capturedAt") # rename and convert to timestamp datatype
  .withColumn("capturedAt", unix_timestamp( col("capturedAt"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp") )
  .orderBy( col("capturedAt"), col("site") )    # sort our records
  .cache()                                      # Cache the expensive operation
)
# materialize the cache
pageviewsDF.count()

In [0]:
display(pageviewsDF)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) date_format()

Today, we want to aggregate all the data **by the day of week** (Monday, Tuesday, Wednesday)...

...and then **sum all the requests**.

Our goal is to see **which day of the week** has the most traffic.

On of the functions that can help us with this the operation `date_format(..)` from the `functions` package.

If you recall from our review of `unix_timestamp(..)` Spark uses the <a href="https://docs.oracle.com/javase/8/docs/api/java/text/SimpleDateFormat.html" target="_blank">SimpleDateFormat</a> from the Java API for parsing (and formatting).

In [0]:
# Create a new DataFrame
byDayOfWeekDF = (pageviewsDF
  .groupBy( date_format( col("capturedAt"), "E") )         # format as Mon, Tue and then aggregate
  .sum()                                                   # produce the sum of all records
  .select( col("date_format(capturedAt, E)").alias("dow"), # rename to "dow"
           col("sum(requests)").alias("total"))            # rename to "total"
  .orderBy( col("dow") )                                   # sort by "dow" MTWTFSS
)

With that's done, let's look at the result.

But not just as a list of tables...

Sometimes you miss important details when you are looking at just numbers.

Let's try a bar graph - all you have to do is click on the graph icon below your results.

In [0]:
display(byDayOfWeekDF)

What's wrong with this graph?

What if we could change the labels from "Mon" to "1-Mon" and "Tue" to "2-Tue"?

Would that fix the problem? Sure...

What API call(s) would solve that problem...

Well there isn't one. We'll just have to create our own...

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) User Defined Functions (UDF)

As you've seen, `DataFrames` can do anything!

Actually they can't.

There will always be the case for a transformation that cannot be accomplished with the provided functions.

To address those cases, Apache Spark provides for **User Defined Functions** or **UDF** for short.

However, they come with a cost...
* **UDFs cannot be optimized** by the Catalyst Optimizer - someday, maybe, but for now it has no insight to your code.
* The function **has to be serialized** and sent out to the executors - this is a one-time cost, but a cost just the same.
* If you are not careful, you could **serialize the whole world**.
* In the case of Python, there is even more over head - we have to **spin up a Python interpreter** on every Executor to run your Lambda.

Let's start with our function...

In [0]:
def mapDayOfWeek(day):
  _dow = {"Mon": "1", "Tue": "2", "Wed": "3", "Thu": "4", "Fri": "5", "Sat": "6", "Sun": "7"}
  
  n = _dow.get(day)
  if n:
    return n + "-" + day
  else:
    return "UNKNOWN"

And now we can test our function...

In [0]:
assert "1-Mon" == mapDayOfWeek("Mon")
assert "2-Tue" == mapDayOfWeek("Tue")
assert "3-Wed" == mapDayOfWeek("Wed")
assert "4-Thu" == mapDayOfWeek("Thu")
assert "5-Fri" == mapDayOfWeek("Fri")
assert "6-Sat" == mapDayOfWeek("Sat")
assert "7-Sun" == mapDayOfWeek("Sun")
assert "UNKNOWN" == mapDayOfWeek("Xxx")

Great, it works! Now define a UDF that wraps this function:

In [0]:
prependNumberUDF = udf(mapDayOfWeek)

At this point, we have told Spark that we want that **function to be serialized**...

and sent out **to each executor**.

**We can test it** as a UDF with a simple query...

In [0]:
(byDayOfWeekDF
   .select( prependNumberUDF( col("dow")) )
   .show()
)

Our UDF looks like **it's working**. 

Next, let's **apply the UDF** and also **order the x axis** from Mon -> Sun

Once it's done executing, we can **render our bar graph**.

In [0]:
display(
  byDayOfWeekDF
    .select( prependNumberUDF(col("dow")).alias("Day Of Week"), col("total") )
    .orderBy("Day Of Week")
)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) I Lied

> What API call(s) would solve that problem...<br/>
> Well there isn't one. We'll just have to create our own...

The truth is that **there is already a function** to do exactly what we want.

Before you go and create your own UDF, check, double check, triple check to make sure it doesn't exist.

Remember...
* UDFs induce a performance hit.
* Big ones in the case of Python.
* The Catalyst Optimizer cannot optimize it.
* And you are re-inventing the wheel.

In this case, the solution to our problem is the operation `date_format(..)` from the `...sql.functions`.

In [0]:
display(
  pageviewsDF
    .groupBy( date_format(col("capturedAt"), "u-E").alias("Day Of Week") )
    .sum()
    .orderBy(col("Day Of Week"))
)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Conclusions?
  
So what can we infer from this data?
  
Remember... we draw our conclusions first and **then** back 'em up with the data.
0. Why does Monday have more records than any day of the week?
0. Why does the weekend have less records than the rest of the week?
0. What can we conclude about usage from Friday to Saturday to Sunday?
0. Is there a correlation between that and Sunday to Monday?

See also <a href="https://docs.azuredatabricks.net/spark/latest/spark-sql/udaf-scala.html" target="_blank">User Defined Aggregate Functions</a>

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) What About Mondays?

Something is up with Mondays.

But is the problem unique to Mobile or Desktop?

To answer this, we can fork the `DataFrame`.

One for Mobile, another for Desktop.

In [0]:
mobileDF = (pageviewsDF
    .filter(col("site") == "mobile")
    .groupBy( date_format(col("capturedAt"), "u-E").alias("Day Of Week") )
    .sum()
    .withColumnRenamed("sum(requests)", "Mobile Total")
    .orderBy(col("Day Of Week"))
)
desktopDF = (pageviewsDF
    .filter(col("site") == "desktop")
    .groupBy( date_format(col("capturedAt"), "u-E").alias("Day Of Week") )
    .sum()
    .withColumnRenamed("sum(requests)", "Desktop Total")
    .orderBy(col("Day Of Week"))
)
# Cache and materialize
mobileDF.cache().count()
desktopDF.cache().count()

Now we can create two graphs, one for Mobile and another for Desktop.

But more realistically, I can better see the scale between the two if I could view them in one graph.

But even more importantly, one graph is really just a semi-contrived reason to demonstrate a `join(..)` operation.

In [0]:
display(mobileDF)

In [0]:
display(desktopDF)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) join(..)

If you a familiar with SQL joins then `DataFrame.join(..)` should be pretty strait forward.

We start with the left side - in this example `desktopDF`.

Then join the right side - in this example `mobileDF`.

The tricky part is the join condition.

We want all records to align by the **Day Of Week** column.

The problem is that if we used `$"Day Of Week"` or `col("Day Of Week")` the `DataFrame` cannot tell which source we are referring to...
* `desktopDF`'s version of **Day Of Week** or
* `mobileDF`'s version of **Day Of Week**

If you recall from our discussions on the `Column` class, one option was to use the `DataFrame` to return an instance of a `Column`.

For example.

In [0]:
columnA = desktopDF["Day Of Week"]
columnB = mobileDF["Day Of Week"]

print(columnA) # Python hack to print the data type
print(columnB)

And now we can put it all together...

In [0]:
tempDF = desktopDF.join(mobileDF, desktopDF["Day Of Week"] == mobileDF["Day Of Week"])

display(tempDF)

As we can see above, we now have the two `DataFrames` "joined" into a single one.

However, if you notice, we have two **Day Of Week** columns.

This will just create headaches later.

Let's drop `desktopDF`'s copy of **Day Of Week** using the same technique we used in the join.

And while we are at it, use a `select(..)` transformation to rearrange the columns.

In [0]:
joinedDF = (tempDF
  .drop(desktopDF["Day Of Week"])
  .select(col("Day Of Week"), col("Desktop Total"), col("Mobile Total"))
)
display(joinedDF)

Of course, there is always more than one way to solve the same problem.

In this case we can use an equi-join:

In [0]:
altDF = desktopDF.join(mobileDF, "Day Of Week")

display(altDF)

And just to wrap it up, plot the graph.

It should only show one of the two numbers.

Open the **Plot Options..** and...
* Set the **Keys** to **Day Of Week**.
* Set the **Values** to both **Desktop Total** AND **Mobile Total**.
* Hint: You can drag and drop the values.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) More Conclusions?
  
Now, what can we infer from this data?
  
0. What is the difference between the weekend numbers (F/S/S) for Mobile vs Desktop?
0. What would explain the difference in weekend numbers for the two sites?
0. How does the weekend numbers compare to Mondays? For Desktop? For Mobile?
0. What conclusion can we draw about Mondays?

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [0]:
%run "../Includes/Classroom-Cleanup"

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/labs.png) Data Frames Lab #4

Unlike the previous labs, this next lab does not have an emphasis on UDFs or Joins.

However, you will have to draw on everything you have learned so far in order to answer the question...

What's up with Monday?

Go ahead and open the notebook [Introduction to DataFrames, Lab #4]($./Intro To DF Part 4 Lab) and complete the exercises.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>